In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [4]:
# load data from google drive
train = pd.read_csv('/content/gdrive/Shared drives/Predictive Analytics Project/Data/Original Dataset/train.csv')
resources = pd.read_csv('/content/gdrive/Shared drives/Predictive Analytics Project/Data/Original Dataset/resources.csv')
test = pd.read_csv('/content/gdrive/Shared drives/Predictive Analytics Project/Data/Original Dataset/test.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
y_train=train['project_is_approved'].values
X_train=train.drop(['project_is_approved'],axis=1)
X_test= test

In [0]:
UNKNOWN_WORD = "_UNK_"
END_WORD = "_END_"
NAN_WORD = "_NAN_"
CLASSES = ["project_is_approved"]

In [0]:
# save y train data 
import pickle
def save_obj(obj, fname):
    with open(fname, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

save_obj(y_train,'y_train.pkl')
# load_obj('y_train.pkl')  -- load y

### Merging columns

In [0]:
X_train["application_text"] = X_train["project_title"].map(str) +\
                        X_train["project_essay_1"].map(str) +\
                        X_train["project_essay_2"].map(str) + \
                        X_train["project_essay_3"].map(str) + \
                        X_train["project_essay_4"].map(str)
X_train["application_text"] = X_train["application_text"].fillna(NAN_WORD)

In [0]:
X_test["application_text"] = X_test["project_title"].map(str) +\
                        X_test["project_essay_1"].map(str) +\
                        X_test["project_essay_2"].map(str) + \
                        X_test["project_essay_3"].map(str) + \
                        X_test["project_essay_4"].map(str)
X_test["application_text"] = X_test["application_text"].fillna(NAN_WORD)

### decontraction

In [0]:
## decontracted
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

### removing stopwords, puncutation and tags

In [0]:
## removing stopwords punctuation

stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [15]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")


def text_process(mess):

    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
from tqdm import tqdm_notebook as tqdm

preprocessed_essays_X_train = []
# tqdm is for printing the status bar

for sentance in tqdm(X_train["application_text"]):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
    preprocessed_essays_X_train.append(sent.lower().strip())

In [26]:
preprocessed_essays_X_test = []
# tqdm is for printing the status bar
for sentance in tqdm(X_test["application_text"]):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
    preprocessed_essays_X_test.append(sent.lower().strip())

### tokenization

In [27]:
## tokenize
import nltk 
nltk.download("punkt")
def tokenize_sentences(sentences, words_dict):
    tokenized_sentences = []
    for sentence in tqdm(sentences):
        if hasattr(sentence, "decode"):
            sentence = sentence.decode("utf-8")
        tokens = nltk.tokenize.word_tokenize(sentence)
        result = []
        for word in tokens:
            word = word.lower()
            if word not in words_dict:
                words_dict[word] = len(words_dict)
            word_index = words_dict[word]
            result.append(word_index)
        tokenized_sentences.append(result)
    return tokenized_sentences, words_dict

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [28]:
print("Loading data...")
list_sentences_X_train = np.asarray(preprocessed_essays_X_train)
list_sentences_X_test = np.asarray(preprocessed_essays_X_test)
print("Done.")

Loading data...
Done.


In [29]:
print("Tokenizing sentences in train set...")
tokenized_sentences_train, words_dict = tokenize_sentences(list_sentences_X_train, {})
print("Tokenizing sentences in test set...")
tokenized_sentences_test, words_dict = tokenize_sentences(list_sentences_X_test, words_dict)

Tokenizing sentences in train set...


Tokenizing sentences in test set...


In [30]:
words_dict[UNKNOWN_WORD] = len(words_dict)
print(tokenized_sentences_train[1])
print(words_dict[UNKNOWN_WORD])

[110, 111, 112, 113, 114, 14, 115, 116, 14, 117, 118, 119, 5, 120, 121, 122, 123, 124, 14, 118, 125, 5, 126, 127, 128, 129, 130, 131, 132, 22, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 23, 64, 146, 147, 148, 64, 149, 116, 150, 151, 95, 5, 152, 153, 154, 155, 156, 157, 158, 159, 160, 82, 117, 118, 5, 23, 161, 162, 73, 163, 164, 165, 166, 167, 112, 168, 169, 114, 170, 171, 14, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 112, 168, 190, 173, 191, 192, 193, 194, 134, 195, 196, 197, 198, 199, 14, 31, 109]
117699


### TF-IDF


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=10)
text_tfidf = vectorizer.fit_transform(preprocessed_essays_X_train)
print("Shape of matrix after one hot encodig ",text_tfidf.shape)

Shape of matrix after one hot encodig  (182080, 21570)


In [0]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=320, random_state=50)
textidf = svd.fit_transform(text_tfidf)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=10)
text_tfidf = vectorizer.fit_transform(preprocessed_essays_X_test)
print("Shape of matrix after one hot encodig ",text_tfidf.shape)

Shape of matrix after one hot encodig  (78035, 15264)


In [0]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=320, random_state=50)
textidf = svd.fit_transform(text_tfidf)

### word embedding

In [0]:
#embedding

def read_embedding_list(file_path):
    embedding_word_dict = {}
    embedding_list = []
    f = open(file_path, encoding="utf8")
    #f = open(file_path)
    for index, line in enumerate(f):
        if index == 0:
            continue
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            continue
        embedding_list.append(coefs)
        embedding_word_dict[word] = len(embedding_word_dict)
    f.close()
    embedding_list = np.array(embedding_list)
    return embedding_list, embedding_word_dict

def clear_embedding_list(embedding_list, embedding_word_dict, words_dict):
    cleared_embedding_list = []
    cleared_embedding_word_dict = {}

    for word in words_dict:
        if word not in embedding_word_dict:
            continue
        word_id = embedding_word_dict[word]
        row = embedding_list[word_id]
        cleared_embedding_list.append(row)
        cleared_embedding_word_dict[word] = len(cleared_embedding_word_dict)

    return cleared_embedding_list, cleared_embedding_word_dict

def convert_tokens_to_ids(tokenized_sentences, words_list, embedding_word_dict, sentences_length):
    words_train = []

    for sentence in tokenized_sentences:
        current_words = []
        for word_index in sentence:
            word = words_list[word_index]
            word_id = embedding_word_dict.get(word, len(embedding_word_dict) - 2)
            current_words.append(word_id)

        if len(current_words) >= sentences_length:
            current_words = current_words[:sentences_length]
        else:
            current_words += [len(embedding_word_dict) - 1] * (sentences_length - len(current_words))
        words_train.append(current_words)
    return words_train



In [32]:
# Embedding

embedding_path = "/content/gdrive/Shared drives/Predictive Analytics Project/Feature Engineering/crawl-300d-2M.vec"

print("Loading embeddings...")
embedding_list, embedding_word_dict = read_embedding_list(embedding_path)
embedding_size = len(embedding_list[0])
print("Done")



Loading embeddings...
Done


In [0]:
batch_size = 256
recurrent_units = 64
dropout_rate = 0.3 
dense_size = 32
sentences_length = 320
fold_count = 5

In [34]:
print("Preparing data...")
embedding_list, embedding_word_dict = clear_embedding_list(embedding_list, embedding_word_dict, words_dict)

embedding_word_dict[UNKNOWN_WORD] = len(embedding_word_dict)
embedding_list.append([0.] * embedding_size)
embedding_word_dict[END_WORD] = len(embedding_word_dict)
embedding_list.append([-1.] * embedding_size)

embedding_matrix = np.array(embedding_list)

id_to_word = dict((id, word) for word, id in words_dict.items())
train_list_of_token_ids = convert_tokens_to_ids(
    tokenized_sentences_train,
    id_to_word,
    embedding_word_dict,
    sentences_length)
test_list_of_token_ids = convert_tokens_to_ids(
    tokenized_sentences_test,
    id_to_word,
    embedding_word_dict,
    sentences_length)
X_train_text = np.array(train_list_of_token_ids)
X_test_text = np.array(test_list_of_token_ids)
print("Done")

Preparing data...
Done


In [35]:
X_train_text.shape

(182080, 320)

### GloVe

In [0]:
def loadGloveModel(gloveFile):
    
    print ("Loading Glove Model")
    
    f = open(gloveFile,'r', encoding = 'utf8')
    
    model = {}
    
    for line in tqdm(f):
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    
    print ("Done.",len(model)," words loaded!")
    
    return model

In [0]:
model = loadGloveModel('/content/gdrive/Shared drives/Predictive Analytics Project/Feature Engineering/glove.42B.300d.txt')

Loading Glove Model



Done. 1917494  words loaded!


In [0]:
import pickle
with open('glove.42B.300d.txt', 'wb') as f:
    pickle.dump(words_corpus, f)

with open('glove.42B.300d.txt', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

# average Word2Vec
# compute average word2vec for each review.
avg_w2v_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(preprocessed_essays_X_train): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split(): # for each word in a review/sentence
        if word in glove_words:
            vector += model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    avg_w2v_vectors.append(vector)

print(len(avg_w2v_vectors))
print(len(avg_w2v_vectors[0]))

In [0]:
save_obj(X_train_text,'X_train_text.pkl')
save_obj(X_test_text,'X_train_text.pkl')